In [10]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import scipy as sp
from pydataset import data
from env import get_db_url, user, password, host
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

import acquire as acq
import os
directory = os.getcwd()

## Exercises

### 1. Create a new file named model_evaluation.py or model_evaluation.ipynb for these exercises.
### 2. Given the following confusion matrix, evaluate (by hand) the model's performance.
|               | pred dog (+)  | pred not dog (-)  |
|:------------  |-----------:|-----------:|
| actual dog (+)   |         46 |         7  |
| actual not dog (-)   |         13 |         34 |

#### - In the context of this problem, what is a false positive?
#### - In the context of this problem, what is a false negative?
#### - How would you describe this model?

In [15]:
Accuracy = (34 + 46) / (34 + 7 + 13 + 46)
Accuracy
# - In the context of this problem, what is a false positive? Answer-
# I will assume we are predicting to find dog.

# FP: false positive happens when predicted (+) and actual (-) occurs, which in this case = 13
# Type I error 

0.8

In [4]:
# - In the context of this problem, what is a false negative? Answer-
# FN: happens when predicted negative (-) and actul positive (+) occurs, which in this case = 7
# Type II error

In [5]:
# - How would you describe this model? Answer-
# TP: We predicted dog, was dog (46)
# TN: We predicted not-dog, was not-dog (34)
# FP: We predicted dog, was not-dog (13) - Type I Error
# FN: We predicted not-dog, was dog (7) - Type II Error
TP = 46
TN = 34
FP = 13
FN = 7

#with an accuracy of 0.8, i think the model did well.

#### 3. You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.
#### Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.
#### Use the predictions dataset and pandas to help answer the following questions:

In [16]:
ducks = pd.read_csv('c3.csv')
ducks.head()

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect
3,No Defect,Defect,Defect,Defect
4,No Defect,No Defect,Defect,No Defect


#### - An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

In [17]:
# True Positive Results

TP_m1 = ducks[(ducks.actual == 'Defect') & (ducks.model1 == 'Defect')]
TP_m1 = TP_m1[['actual', 'model1']]
TP_m1 = TP_m1.shape[0]

TP_m2 = ducks[(ducks.actual == 'Defect') & (ducks.model2 == 'Defect')]
TP_m2 = TP_m2[['actual', 'model2']]
TP_m2 = TP_m2.shape[0]

TP_m3 = ducks[(ducks.actual == 'Defect') & (ducks.model3 == 'Defect')]
TP_m3 = TP_m3[['actual', 'model3']]
TP_m3 = TP_m3.shape[0]

In [18]:
# True Negatives

TN_m1 = ducks[(ducks.actual == 'No Defect') & (ducks.model1 == 'No Defect')]
TN_m1 = TN_m1[['actual', 'model1']]
TN_m1 = TN_m1.shape[0]

TN_m2 = ducks[(ducks.actual == 'No Defect') & (ducks.model2 == 'No Defect')]
TN_m2 = TN_m2[['actual', 'model2']]
TN_m2 = TN_m2.shape[0]

TN_m3 = ducks[(ducks.actual == 'No Defect') & (ducks.model3 == 'No Defect')]
TN_m3 = TN_m3[['actual', 'model3']]
TN_m3 = TN_m3.shape[0]

In [19]:
# False Positives

FP_m1 = ducks[(ducks.actual == 'No Defect') & (ducks.model1 == 'Defect')]
FP_m1 = FP_m1[['actual', 'model1']]
FP_m1 = FP_m1.shape[0]

FP_m2 = ducks[(ducks.actual == 'No Defect') & (ducks.model2 == 'Defect')]
FP_m2 = FP_m2[['actual', 'model2']]
FP_m2 = FP_m2.shape[0]

FP_m3 = ducks[(ducks.actual == 'No Defect') & (ducks.model3 == 'Defect')]
FP_m3 = FP_m3[['actual', 'model3']]
FP_m3 = FP_m3.shape[0]

In [20]:
# False Negative

FN_m1 = ducks[(ducks.actual == 'Defect') & (ducks.model1 == 'No Defect')]
FN_m1 = FN_m1[['actual', 'model1']]
FN_m1 = FN_m1.shape[0]

FN_m2 = ducks[(ducks.actual == 'Defect') & (ducks.model2 == 'No Defect')]
FN_m2 = FN_m2[['actual', 'model2']]
FN_m2 = FN_m2.shape[0]

FN_m3 = ducks[(ducks.actual == 'Defect') & (ducks.model3 == 'No Defect')]
FN_m3 = FN_m3[['actual', 'model3']]
FN_m3 = FN_m3.shape[0]

#### - Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

#### 4. You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).
#### At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).
#### Several models have already been developed with the data, and you can find their results here.
#### Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions:
#### a. In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?

#### b. Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recommend?

#### c. Suppose you are working on a team that solely deals with cat pictures. Which of these models would you recommend?

#### 5. Follow the links below to read the documentation about each function, then apply those functions to the data from the previous problem.

- sklearn.metrics.accuracy_score
- sklearn.metrics.precision_score
- sklearn.metrics.recall_score
- sklearn.metrics.classification_report